<a href="https://colab.research.google.com/github/PhamVoMinhKhoi/MySQL_Cleaning-EDA/blob/main/B%C3%A1o_c%C3%A1o_tt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cài đặt và import các thư viện cần thiết cho chương trình

In [56]:

!pip install emoji
!pip install pandasql
!pip install sentence-transformers
!pip install nltk spacy numpy
!python -m spacy download en_core_web_sm
!pip install git+https://github.com/boudinfl/pke.git
!pip install unidecode
!pip install Unidecode



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 67.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-l9_rmxfs
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-l9_rmxfs
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done


In [57]:
import emoji
import string
import numpy as np
import pandas as pd
import pandasql as ps
from unidecode import unidecode
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import re
import nltk
from nltk.metrics import edit_distance
import torch
import torch.nn.functional as F

# Đọc dữ liệu

In [58]:
file_path ='/content/Demo data.xlsx'
data = pd.read_excel(file_path)

In [59]:
data.head(5)

,Address,House_number,Street,Quarter
0,"01 đường Trần Lựu, Khu phố 7",NaN,NaN,NaN
1,"01, đường Lương Định Của, Khu phố 1",NaN,NaN,NaN
2,01/11 Khu phố 10,NaN,NaN,NaN
3,"02 đường 04, Khu phố 4",NaN,NaN,NaN
4,"02 Đường 4, Khu phố 4",NaN,NaN,NaN


In [60]:
data.tail(5)

,Address,House_number,Street,Quarter
2952,"Số nhà D3/6, Khu phố 8",NaN,NaN,NaN
2953,"Số nhà G2/10Đ, đường 4, Khu phố 4",NaN,NaN,NaN
2954,"Số nhà 1/4A trần não, Khu phố 1",NaN,NaN,NaN
2955,"Số nhà 1/4A trần não, Khu phố 3",NaN,NaN,NaN
2956,"T4 Đường 4, Khu phố 4",NaN,NaN,NaN


In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2957 entries, 0 to 2956
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Address       2954 non-null   object 
 1   House_number  0 non-null      float64
 2   Street        0 non-null      float64
 3   Quarter       0 non-null      float64
dtypes: float64(3), object(1)
memory usage: 92.5+ KB


# Làm sạch dữ liệu, tách số nhà và khu phố

In [62]:
df = data.copy()

In [63]:
df['Address_process'] = df['Address']

In [64]:
df = df.reindex(columns=['Address','Address_process', 'House_number', 'Street', 'Quarter'])
df

,Address,Address_process,House_number,Street,Quarter
0,"01 đường Trần Lựu, Khu phố 7","01 đường Trần Lựu, Khu phố 7",NaN,NaN,NaN
1,"01, đường Lương Định Của, Khu phố 1","01, đường Lương Định Của, Khu phố 1",NaN,NaN,NaN
2,01/11 Khu phố 10,01/11 Khu phố 10,NaN,NaN,NaN
3,"02 đường 04, Khu phố 4","02 đường 04, Khu phố 4",NaN,NaN,NaN
4,"02 Đường 4, Khu phố 4","02 Đường 4, Khu phố 4",NaN,NaN,NaN
...,...,...,...,...,...
2952,"Số nhà D3/6, Khu phố 8","Số nhà D3/6, Khu phố 8",NaN,NaN,NaN
2953,"Số nhà G2/10Đ, đường 4, Khu phố 4","Số nhà G2/10Đ, đường 4, Khu phố 4",NaN,NaN,NaN
2954,"Số nhà 1/4A trần não, Khu phố 1","Số nhà 1/4A trần não, Khu phố 1",NaN,NaN,NaN
2955,"Số nhà 1/4A trần não, Khu phố 3","Số nhà 1/4A trần não, Khu phố 3",NaN,NaN,NaN


In [65]:
# Tiến hành viết hoa tất cả dữ liệu trong bản
df['Address_process'] = df['Address_process'].str.upper()

In [66]:
df

,Address,Address_process,House_number,Street,Quarter
0,"01 đường Trần Lựu, Khu phố 7","01 ĐƯỜNG TRẦN LỰU, KHU PHỐ 7",NaN,NaN,NaN
1,"01, đường Lương Định Của, Khu phố 1","01, ĐƯỜNG LƯƠNG ĐỊNH CỦA, KHU PHỐ 1",NaN,NaN,NaN
2,01/11 Khu phố 10,01/11 KHU PHỐ 10,NaN,NaN,NaN
3,"02 đường 04, Khu phố 4","02 ĐƯỜNG 04, KHU PHỐ 4",NaN,NaN,NaN
4,"02 Đường 4, Khu phố 4","02 ĐƯỜNG 4, KHU PHỐ 4",NaN,NaN,NaN
...,...,...,...,...,...
2952,"Số nhà D3/6, Khu phố 8","SỐ NHÀ D3/6, KHU PHỐ 8",NaN,NaN,NaN
2953,"Số nhà G2/10Đ, đường 4, Khu phố 4","SỐ NHÀ G2/10Đ, ĐƯỜNG 4, KHU PHỐ 4",NaN,NaN,NaN
2954,"Số nhà 1/4A trần não, Khu phố 1","SỐ NHÀ 1/4A TRẦN NÃO, KHU PHỐ 1",NaN,NaN,NaN
2955,"Số nhà 1/4A trần não, Khu phố 3","SỐ NHÀ 1/4A TRẦN NÃO, KHU PHỐ 3",NaN,NaN,NaN


In [67]:
# xóa dấu "," trong cột Address_process
df['Address_process'] = df['Address_process'].str.replace(",", "", regex=False)

In [68]:
# đổi 'đường số' thành 'đường'
df['Address_process'] = df['Address_process'].str.replace("ĐƯỜNG SỐ", "ĐƯỜNG", regex=False)

In [69]:
df

,Address,Address_process,House_number,Street,Quarter
0,"01 đường Trần Lựu, Khu phố 7",01 ĐƯỜNG TRẦN LỰU KHU PHỐ 7,NaN,NaN,NaN
1,"01, đường Lương Định Của, Khu phố 1",01 ĐƯỜNG LƯƠNG ĐỊNH CỦA KHU PHỐ 1,NaN,NaN,NaN
2,01/11 Khu phố 10,01/11 KHU PHỐ 10,NaN,NaN,NaN
3,"02 đường 04, Khu phố 4",02 ĐƯỜNG 04 KHU PHỐ 4,NaN,NaN,NaN
4,"02 Đường 4, Khu phố 4",02 ĐƯỜNG 4 KHU PHỐ 4,NaN,NaN,NaN
...,...,...,...,...,...
2952,"Số nhà D3/6, Khu phố 8",SỐ NHÀ D3/6 KHU PHỐ 8,NaN,NaN,NaN
2953,"Số nhà G2/10Đ, đường 4, Khu phố 4",SỐ NHÀ G2/10Đ ĐƯỜNG 4 KHU PHỐ 4,NaN,NaN,NaN
2954,"Số nhà 1/4A trần não, Khu phố 1",SỐ NHÀ 1/4A TRẦN NÃO KHU PHỐ 1,NaN,NaN,NaN
2955,"Số nhà 1/4A trần não, Khu phố 3",SỐ NHÀ 1/4A TRẦN NÃO KHU PHỐ 3,NaN,NaN,NaN


In [70]:
# 'Số nhà' thành 'số'

In [71]:
df['Address_process'] = df['Address_process'].str.replace("SỐ NHÀ", "SỐ", regex=False)

In [72]:
# Thếm số vào đầu của mỗi giá trị trong cột
def add_so_to_text(text):
    # Kiểm tra xem chuỗi có bắt đầu bằng "SỐ" hay không
    if not str(text).startswith("SỐ") and not str(text).startswith("KHU"):
        return "SỐ " + str(text)
    return text

In [73]:
df['Address_process'] = df['Address_process'].apply(add_so_to_text)

In [74]:
df.head()

,Address,Address_process,House_number,Street,Quarter
0,"01 đường Trần Lựu, Khu phố 7",SỐ 01 ĐƯỜNG TRẦN LỰU KHU PHỐ 7,NaN,NaN,NaN
1,"01, đường Lương Định Của, Khu phố 1",SỐ 01 ĐƯỜNG LƯƠNG ĐỊNH CỦA KHU PHỐ 1,NaN,NaN,NaN
2,01/11 Khu phố 10,SỐ 01/11 KHU PHỐ 10,NaN,NaN,NaN
3,"02 đường 04, Khu phố 4",SỐ 02 ĐƯỜNG 04 KHU PHỐ 4,NaN,NaN,NaN
4,"02 Đường 4, Khu phố 4",SỐ 02 ĐƯỜNG 4 KHU PHỐ 4,NaN,NaN,NaN


In [75]:
df.tail()

,Address,Address_process,House_number,Street,Quarter
2952,"Số nhà D3/6, Khu phố 8",SỐ D3/6 KHU PHỐ 8,NaN,NaN,NaN
2953,"Số nhà G2/10Đ, đường 4, Khu phố 4",SỐ G2/10Đ ĐƯỜNG 4 KHU PHỐ 4,NaN,NaN,NaN
2954,"Số nhà 1/4A trần não, Khu phố 1",SỐ 1/4A TRẦN NÃO KHU PHỐ 1,NaN,NaN,NaN
2955,"Số nhà 1/4A trần não, Khu phố 3",SỐ 1/4A TRẦN NÃO KHU PHỐ 3,NaN,NaN,NaN
2956,"T4 Đường 4, Khu phố 4",SỐ T4 ĐƯỜNG 4 KHU PHỐ 4,NaN,NaN,NaN


In [76]:
def get_house_number(text):
    # Sử dụng biểu thức chính quy để tìm kiếm chuỗi "SỐ " theo sau bởi một hoặc nhiều ký tự không phải là dấu cách
    match = re.search(r"SỐ\s([^\s]+)", text)

    if match:
        # Nếu tìm thấy, lấy số nhà từ nhóm đầu tiên trong kết quả match (sau "SỐ ")
        h_num = match.group(1)

        # Cập nhật chuỗi văn bản bằng cách loại bỏ phần "SỐ " và số nhà đã tìm được
        updated_text = text.replace("SỐ " + h_num, "", 1).strip()

        # Trả về chuỗi văn bản đã cập nhật (loại bỏ "SỐ <số nhà>") và số nhà tìm được
        return updated_text, h_num

    # Nếu không tìm thấy, trả về chuỗi văn bản ban đầu và giá trị None cho số nhà
    return text, None

In [77]:
df.head()

,Address,Address_process,House_number,Street,Quarter
0,"01 đường Trần Lựu, Khu phố 7",SỐ 01 ĐƯỜNG TRẦN LỰU KHU PHỐ 7,NaN,NaN,NaN
1,"01, đường Lương Định Của, Khu phố 1",SỐ 01 ĐƯỜNG LƯƠNG ĐỊNH CỦA KHU PHỐ 1,NaN,NaN,NaN
2,01/11 Khu phố 10,SỐ 01/11 KHU PHỐ 10,NaN,NaN,NaN
3,"02 đường 04, Khu phố 4",SỐ 02 ĐƯỜNG 04 KHU PHỐ 4,NaN,NaN,NaN
4,"02 Đường 4, Khu phố 4",SỐ 02 ĐƯỜNG 4 KHU PHỐ 4,NaN,NaN,NaN


In [78]:
df[['Address_process', 'House_number']] = df['Address_process'].apply(lambda x: pd.Series(get_house_number(x)))
# Kiểm tra kết quả
df[['Address_process', 'House_number']].head()

,Address_process,House_number
0,ĐƯỜNG TRẦN LỰU KHU PHỐ 7,01
1,ĐƯỜNG LƯƠNG ĐỊNH CỦA KHU PHỐ 1,01
2,KHU PHỐ 10,01/11
3,ĐƯỜNG 04 KHU PHỐ 4,02
4,ĐƯỜNG 4 KHU PHỐ 4,02


In [79]:
df.head()

,Address,Address_process,House_number,Street,Quarter
0,"01 đường Trần Lựu, Khu phố 7",ĐƯỜNG TRẦN LỰU KHU PHỐ 7,01,NaN,NaN
1,"01, đường Lương Định Của, Khu phố 1",ĐƯỜNG LƯƠNG ĐỊNH CỦA KHU PHỐ 1,01,NaN,NaN
2,01/11 Khu phố 10,KHU PHỐ 10,01/11,NaN,NaN
3,"02 đường 04, Khu phố 4",ĐƯỜNG 04 KHU PHỐ 4,02,NaN,NaN
4,"02 Đường 4, Khu phố 4",ĐƯỜNG 4 KHU PHỐ 4,02,NaN,NaN


In [80]:
def get_quarter_and_update_text(text):
    match = re.search(r"(KHU PHỐ \d+)", text)
    if match:
        quarter = match.group(1)  # Lấy cụm "KHU PHỐ"
        updated_text = text.replace(quarter, '',1).strip()  # Xóa "KHU PHỐ" khỏi văn bản
        return updated_text, quarter  # Trả về văn bản đã cập nhật và cụm
    return text, None  # Nếu không tìm thấy, trả về văn bản gốc và None

In [81]:
df[['Address_process', 'Quarter']] = df['Address_process'].apply(lambda x: pd.Series(get_quarter_and_update_text(x)))

In [82]:
df.head()

,Address,Address_process,House_number,Street,Quarter
0,"01 đường Trần Lựu, Khu phố 7",ĐƯỜNG TRẦN LỰU,01,NaN,KHU PHỐ 7
1,"01, đường Lương Định Của, Khu phố 1",ĐƯỜNG LƯƠNG ĐỊNH CỦA,01,NaN,KHU PHỐ 1
2,01/11 Khu phố 10,,01/11,NaN,KHU PHỐ 10
3,"02 đường 04, Khu phố 4",ĐƯỜNG 04,02,NaN,KHU PHỐ 4
4,"02 Đường 4, Khu phố 4",ĐƯỜNG 4,02,NaN,KHU PHỐ 4


In [83]:
# loại bỏ khoảng trắng thừa 2 bên giá trị
for col in df.columns:
    for i in range(len(df)):
        if isinstance(df.at[i, col], str):  # Kiểm tra nếu giá trị là chuỗi
            df.at[i, col] = df.at[i, col].strip()

In [84]:
df.head()

,Address,Address_process,House_number,Street,Quarter
0,"01 đường Trần Lựu, Khu phố 7",ĐƯỜNG TRẦN LỰU,01,NaN,KHU PHỐ 7
1,"01, đường Lương Định Của, Khu phố 1",ĐƯỜNG LƯƠNG ĐỊNH CỦA,01,NaN,KHU PHỐ 1
2,01/11 Khu phố 10,,01/11,NaN,KHU PHỐ 10
3,"02 đường 04, Khu phố 4",ĐƯỜNG 04,02,NaN,KHU PHỐ 4
4,"02 Đường 4, Khu phố 4",ĐƯỜNG 4,02,NaN,KHU PHỐ 4


# Nhận diện tên đường

In [85]:
df_address = df.copy()

In [86]:
df_address

,Address,Address_process,House_number,Street,Quarter
0,"01 đường Trần Lựu, Khu phố 7",ĐƯỜNG TRẦN LỰU,01,NaN,KHU PHỐ 7
1,"01, đường Lương Định Của, Khu phố 1",ĐƯỜNG LƯƠNG ĐỊNH CỦA,01,NaN,KHU PHỐ 1
2,01/11 Khu phố 10,,01/11,NaN,KHU PHỐ 10
3,"02 đường 04, Khu phố 4",ĐƯỜNG 04,02,NaN,KHU PHỐ 4
4,"02 Đường 4, Khu phố 4",ĐƯỜNG 4,02,NaN,KHU PHỐ 4
...,...,...,...,...,...
2952,"Số nhà D3/6, Khu phố 8",,D3/6,NaN,KHU PHỐ 8
2953,"Số nhà G2/10Đ, đường 4, Khu phố 4",ĐƯỜNG 4,G2/10Đ,NaN,KHU PHỐ 4
2954,"Số nhà 1/4A trần não, Khu phố 1",TRẦN NÃO,1/4A,NaN,KHU PHỐ 1
2955,"Số nhà 1/4A trần não, Khu phố 3",TRẦN NÃO,1/4A,NaN,KHU PHỐ 3


In [87]:
df_street_name = pd.read_excel('/content/Street.xlsx')

In [88]:
df_street_name.head()

,Street
0,Đường 1
1,Đường 10
2,Đường 11
3,Đường 12
4,Đường 13


In [89]:
df_street = df_street_name.copy()

In [90]:
df_street.head()

,Street
0,Đường 1
1,Đường 10
2,Đường 11
3,Đường 12
4,Đường 13


In [91]:
df_street['Street']=df_street['Street'].str.upper()

In [92]:
df_street

,Street
0,ĐƯỜNG 1
1,ĐƯỜNG 10
2,ĐƯỜNG 11
3,ĐƯỜNG 12
4,ĐƯỜNG 13
...,...
64,XA LỘ HÀ NỘI
65,TRẦN LỰU
66,ĐƯỜNG 11A
67,ĐƯỜNG 30B


In [93]:
# Xóa dấu tiếng Việt
df_street['Street_Unidecode'] = df_street['Street'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)
df_address['Address_Unidecode'] = df_address['Address_process'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)

In [94]:
df_street.head()

,Street,Street_Unidecode
0,ĐƯỜNG 1,DUONG 1
1,ĐƯỜNG 10,DUONG 10
2,ĐƯỜNG 11,DUONG 11
3,ĐƯỜNG 12,DUONG 12
4,ĐƯỜNG 13,DUONG 13


In [95]:
df_address.head()

,Address,Address_process,House_number,Street,Quarter,Address_Unidecode
0,"01 đường Trần Lựu, Khu phố 7",ĐƯỜNG TRẦN LỰU,01,NaN,KHU PHỐ 7,DUONG TRAN LUU
1,"01, đường Lương Định Của, Khu phố 1",ĐƯỜNG LƯƠNG ĐỊNH CỦA,01,NaN,KHU PHỐ 1,DUONG LUONG DINH CUA
2,01/11 Khu phố 10,,01/11,NaN,KHU PHỐ 10,
3,"02 đường 04, Khu phố 4",ĐƯỜNG 04,02,NaN,KHU PHỐ 4,DUONG 04
4,"02 Đường 4, Khu phố 4",ĐƯỜNG 4,02,NaN,KHU PHỐ 4,DUONG 4


In [96]:
# Chuẩn bị dữ liệu cho Word2Vec
sentences = df_street['Street_Unidecode'].apply(lambda x: x.split()).tolist()

# Huấn luyện mô hình Word2Vec
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Hàm tìm giá trị Street tương ứng
def get_best_match(address):
    # Tính embedding cho address, xử lý trường hợp không tìm thấy từ
    address_words = [word for word in address.split() if word in model.wv]
    if not address_words:  # Nếu không có từ nào trong vocabulary
        return None  # Hoặc một giá trị mặc định khác
    address_vec = np.mean([model.wv[word] for word in address_words], axis=0)
    address_vec = address_vec.reshape(1, -1)  # Reshape thành 2D array

    best_match = None
    highest_similarity = -1

    for _, row in df_street.iterrows():
        # Tính embedding cho street, xử lý trường hợp không tìm thấy từ
        street_words = [word for word in row['Street_Unidecode'].split() if word in model.wv]
        if not street_words:  # Nếu không có từ nào trong vocabulary
            continue  # Bỏ qua dòng này
        street_vec = np.mean([model.wv[word] for word in street_words], axis=0)
        street_vec = street_vec.reshape(1, -1)  # Reshape thành 2D array

        # Tính cosine similarity
        similarity = cosine_similarity(address_vec, street_vec)[0][0]

        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = row['Street']

    return best_match

# Áp dụng hàm cho mỗi giá trị trong Address_Unidecode
df_address['Street'] = df_address['Address_Unidecode'].apply(get_best_match)

# Xem kết quả
df_address[['Address_Unidecode', 'Street']].head()

,Address_Unidecode,Street
0,DUONG TRAN LUU,TRẦN LỰU
1,DUONG LUONG DINH CUA,LƯƠNG ĐỊNH CỦA
2,,None
3,DUONG 04,ĐƯỜNG 46
4,DUONG 4,ĐƯỜNG 4


In [97]:
df_address.head()

,Address,Address_process,House_number,Street,Quarter,Address_Unidecode
0,"01 đường Trần Lựu, Khu phố 7",ĐƯỜNG TRẦN LỰU,01,TRẦN LỰU,KHU PHỐ 7,DUONG TRAN LUU
1,"01, đường Lương Định Của, Khu phố 1",ĐƯỜNG LƯƠNG ĐỊNH CỦA,01,LƯƠNG ĐỊNH CỦA,KHU PHỐ 1,DUONG LUONG DINH CUA
2,01/11 Khu phố 10,,01/11,None,KHU PHỐ 10,
3,"02 đường 04, Khu phố 4",ĐƯỜNG 04,02,ĐƯỜNG 46,KHU PHỐ 4,DUONG 04
4,"02 Đường 4, Khu phố 4",ĐƯỜNG 4,02,ĐƯỜNG 4,KHU PHỐ 4,DUONG 4


In [98]:
df_address.to_excel('Address_2.xlsx', index=False)

In [99]:
df_true_value = pd.read_excel('/content/Street.xlsx', sheet_name='Sheet2')

In [100]:
df_true_value

,True value,Address_Unidecode
0,TRẦN LỰU,DUONG TRAN LUU
1,LƯƠNG ĐỊNH CỦA,DUONG LUONG DINH CUA
2,NaN,NaN
3,ĐƯỜNG 4,DUONG 04
4,ĐƯỜNG 4,DUONG 4
...,...,...
2952,NaN,NaN
2953,ĐƯỜNG 4,DUONG 4
2954,TRẦN NÃO,TRAN NAO
2955,TRẦN NÃO,TRAN NAO


In [101]:
# Gộp hai dataframe dựa trên cột Address_Unicode
df_merged = pd.merge(df_address, df_true_value, left_on='Address_Unidecode', right_on='Address_Unidecode', how='left')

# Bây giờ so sánh các cột 'Street' và 'True value' trong DataFrame đã gộp
df_merged['Correct'] = df_merged['Street'] == df_merged['True value']

# Tính độ chính xác (accuracy) sử dụng DataFrame đã gộp
accuracy = df_merged['Correct'].mean() * 100
accuracy

95.37661322993229

In [102]:
df_merged

,Address,Address_process,House_number,Street,Quarter,Address_Unidecode,True value,Correct
0,"01 đường Trần Lựu, Khu phố 7",ĐƯỜNG TRẦN LỰU,01,TRẦN LỰU,KHU PHỐ 7,DUONG TRAN LUU,TRẦN LỰU,True
1,"01 đường Trần Lựu, Khu phố 7",ĐƯỜNG TRẦN LỰU,01,TRẦN LỰU,KHU PHỐ 7,DUONG TRAN LUU,TRẦN LỰU,True
2,"01 đường Trần Lựu, Khu phố 7",ĐƯỜNG TRẦN LỰU,01,TRẦN LỰU,KHU PHỐ 7,DUONG TRAN LUU,TRẦN LỰU,True
3,"01, đường Lương Định Của, Khu phố 1",ĐƯỜNG LƯƠNG ĐỊNH CỦA,01,LƯƠNG ĐỊNH CỦA,KHU PHỐ 1,DUONG LUONG DINH CUA,LƯƠNG ĐỊNH CỦA,True
4,"01, đường Lương Định Của, Khu phố 1",ĐƯỜNG LƯƠNG ĐỊNH CỦA,01,LƯƠNG ĐỊNH CỦA,KHU PHỐ 1,DUONG LUONG DINH CUA,LƯƠNG ĐỊNH CỦA,True
...,...,...,...,...,...,...,...,...
276459,"T4 Đường 4, Khu phố 4",ĐƯỜNG 4,T4,ĐƯỜNG 4,KHU PHỐ 4,DUONG 4,ĐƯỜNG 4,True
276460,"T4 Đường 4, Khu phố 4",ĐƯỜNG 4,T4,ĐƯỜNG 4,KHU PHỐ 4,DUONG 4,ĐƯỜNG 4,True
276461,"T4 Đường 4, Khu phố 4",ĐƯỜNG 4,T4,ĐƯỜNG 4,KHU PHỐ 4,DUONG 4,ĐƯỜNG 4,True
276462,"T4 Đường 4, Khu phố 4",ĐƯỜNG 4,T4,ĐƯỜNG 4,KHU PHỐ 4,DUONG 4,ĐƯỜNG 4,True


In [103]:
df_merged.to_excel('Address_3.xlsx', index=False)